In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)


In [8]:
print(torch.__version__)
print(torchvision.__version__)

1.11.0
0.12.0


In [9]:
#!pip3 install tensorboard
#tensorboard --version

In [10]:
from torch.utils.tensorboard import SummaryWriter

In [13]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels = 6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels = 12,kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.out = nn.Linear(in_features=60,out_features=10)

    def forward(self,t):
        #t = t #第一层输入
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size =2,stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size = 2,stride =2)

        t = t.reshape(-1,12*4*4)
        t = F.relu(self.fc1(t))

        t = F.relu(self.fc2(t))

        t = self.out(t)

        return t


In [12]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/'
    ,train=True
    ,download=True
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [18]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [15]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle = True)


## TansorBoard( Network Graph and Images)

In [17]:
tb = SummaryWriter()
network = Network()
images,lables = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images',grid)
tb.add_graph(network,images)
tb.close()

## 循环的视图

In [21]:
network = Network()

network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer = optim.Adam(network.parameters(),lr = 0.01)

images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images',grid)
tb.add_graph(network,images)

for epoch in range(10):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        images,labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        #------------要将梯度归零 因为pytorch会累加梯度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
#scalar 数字
    tb.add_scalar('loss',total_loss,epoch)
    tb.add_scalar('Number Correcnt',total_correct,epoch)
    tb.add_scalar('Accuracy',total_correct/len(train_set),epoch)
     
    tb.add_histogram('conv1.bias',network.conv1.bias,epoch)
    tb.add_histogram('conv1.weight',network.conv1.weight,epoch)
    tb.add_histogram('conv1.weight.grad',network.conv1.weight.grad,epoch)

    print("epoch:",epoch,"total_correct",total_correct," loss:",total_loss)

tb.close()

epoch: 0 total_correct 44023  loss: 416.4973748922348
epoch: 1 total_correct 49433  loss: 286.7721747159958
epoch: 2 total_correct 50292  loss: 263.8455863445997
epoch: 3 total_correct 50663  loss: 251.5579534471035
epoch: 4 total_correct 50884  loss: 245.59858852624893
epoch: 5 total_correct 51090  loss: 239.03449726104736
epoch: 6 total_correct 51264  loss: 233.15727880597115
epoch: 7 total_correct 51511  loss: 229.62502658367157
epoch: 8 total_correct 51811  loss: 222.43024836480618
epoch: 9 total_correct 51907  loss: 218.03258295357227
